In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import csv

In [3]:
def getSoupFromURL(url):

    try:
        r = requests.get(url)
    except:
        return None

    return BeautifulSoup(r.text, "html5lib")

In [4]:
players_dict = {
    'Ray Allen': 'players/a/allenra02',
    'Reggie Miller': 'players/m/millere01',
    'Jason Terry': 'players/t/terryja01',
    'Paul Pierce': 'players/p/piercpa01',
    'Kyle Korver': 'players/k/korveky01',
    'Jamal Crawford': 'players/c/crawfja01',
    'Vince Carter': 'players/c/cartevi01',
    'Stephen Curry': 'players/c/curryst01',
}
players_list = [
    'Ray Allen',
    'Reggie Miller',
    'Jason Terry',
    'Paul Pierce',
    'Kyle Korver',
    'Jamal Crawford',
    'Vince Carter',
    'Stephen Curry'
]

csv_list = [
    'Ray_Allen',
    'Reggie_Miller',
    'Jason_Terry',
    'Paul_Pierce',
    'Kyle_Korver',
    'Jamal_Crawford',
    'Vince_Carter',
    'Stephen_Curry'
]

In [5]:
BASE_URL = "https://www.basketball-reference.com/"
GAME_LOG_DIR = '/gamelog/'
PLAYER_PROFILE_PAGE = ".html"

In [6]:
def get_data_for_game_log(player, season):
    string_season = str(int(season.split("-")[0]) + 1)
    return BASE_URL + players_dict[player] + GAME_LOG_DIR + string_season

def get_data_for_player(player):
    return BASE_URL + players_dict[player] + PLAYER_PROFILE_PAGE

In [7]:
import time

In [8]:
seasons_for_players = []

for player in players_list:
    soup = getSoupFromURL(get_data_for_player(player))
    time.sleep(1)
    table = soup.find('table', attrs={"class" : "row_summable"})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    seasons = []
    for row in rows:
        season_column = row.find('th', attrs={"data-stat" : "season"})
        season_text = season_column.text
        if season_text not in seasons:
            seasons.append(season_text)
    seasons_for_players.append(seasons)

In [11]:
for player, seasons, file_name in zip(players_list, seasons_for_players, csv_list):
    game_count = 0
    three_point_sum = 0
    list_rows = []
    for season in seasons:
        soup = getSoupFromURL(get_data_for_game_log(player, season))
        time.sleep(1)
        table = soup.find('table', attrs={"class" : "row_summable"})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            three_pointer = row.find('td', attrs={"data-stat" : "fg3"})
            if three_pointer is None:
                continue
            game_count += 1
            three_point_sum += int(three_pointer.text)
            list_rows.append([player, game_count, three_point_sum])
    data_frame = pd.DataFrame(data=list_rows, columns=['PLAYER_NAME', 'GAME_CAREER', 'TOTAL_3MADE'])
    data_frame.to_csv('data/' + file_name + ".csv", header=True)